In [4]:
import subprocess
subprocess.run(['tshark', '-v'])


CompletedProcess(args=['tshark', '-v'], returncode=0)

In [6]:
import os
import subprocess

pcap_folder = r"C:\Users\Sakthi\Downloads\data sales"
pcap_files = [f for f in os.listdir(pcap_folder) if f.endswith(('.pcap', '.pcapng'))]

for filename in pcap_files:
    full_path = os.path.join(pcap_folder, filename)
    print(f"\n Processing File: {filename}")
    print("=" * 60)

    try:
        result = subprocess.run(
            [
                'tshark',
                '-r', full_path,
                '-c', '5',  # Only first 5 packets
                '-T', 'fields',
                '-e', 'frame.number',
                '-e', 'frame.len',
                '-e', '_ws.col.Protocol',
                '-e', '_ws.col.Info'
            ],
            capture_output=True,
            text=True
        )

        if result.returncode == 0:
            print("Number | Length | Protocol | Info")
            print(result.stdout.strip())
        else:
            print(" tshark error:", result.stderr)

    except FileNotFoundError:
        print("tshark is not installed or not in PATH.")



 Processing File: pcap f11.pcapng
Number | Length | Protocol | Info
1	75	SSL	
2	86	TCP	443 â†’ 51593 [ACK] Seq=1 Ack=2 Win=100 Len=0 SLE=1 SRE=2
3	1354	SSL	
4	185	TLSv1.2	Application Data
5	1354	TLSv1.2

 Processing File: pcapf2.pcapng
Number | Length | Protocol | Info
1	140	UDP	443 â†’ 62159 Len=78
2	84	UDP	443 â†’ 62159 Len=22
3	140	UDP	443 â†’ 62159 Len=78
4	96	UDP	62159 â†’ 443 Len=34
5	307	UDP	59693 â†’ 443 Len=245


In [9]:
import os
import subprocess
from collections import Counter
pcap_path = r"C:\Users\Sakthi\Downloads\data sales\pcapf2.pcapng"


cmd = [
    'tshark',
    '-r', pcap_path,
    '-T', 'fields',
    '-e', 'ip.src',
    '-Y', 'ip'  # Filter only IP packets
]

result = subprocess.run(cmd, capture_output=True, text=True)

if result.returncode != 0:
    print(" Error running tshark:", result.stderr)
else:
    src_ips = result.stdout.strip().split('\n')
    ip_counter = Counter(src_ips)

    # Show top 5 source IPs
    print("\n Top 5 Source IPs:")
    for ip, count in ip_counter.most_common(5):
        print(f"{ip}: {count} packets")



 Top 5 Source IPs:
213.186.33.17: 82 packets
10.186.27.162: 71 packets
45.33.87.127: 29 packets
125.56.219.17: 29 packets
172.64.147.34: 18 packets


In [12]:
import subprocess

pcap_path = r"C:\Users\Sakthi\Downloads\data sales\pcapf2.pcapng"


cmd = [
    'tshark',
    '-r', pcap_path,
    '-Y', 'http.request',
    '-T', 'fields',
    '-e', 'http.host',
    '-e', 'http.request.uri'
]

result = subprocess.run(cmd, capture_output=True, text=True)

if result.returncode != 0:
    print(" Error running tshark:", result.stderr)
else:
    print(" Visited URLs:")
    print("=" * 50)
    for line in result.stdout.strip().split('\n'):
        if '\t' in line:
            host, uri = line.split('\t', 1)
            full_url = f"http://{host}{uri}"
            print(f"Host    : {host}")
            print(f"URI     : {uri}")
            print(f"Full URL: {full_url}")
            print("-" * 50)



 Visited URLs:


In [13]:
import subprocess
from collections import Counter

pcap_path = r"C:\Users\Sakthi\Downloads\data sales\pcapf2.pcapng"

cmd = [
    'tshark',
    '-r', pcap_path,
    '-Y', 'dns',
    '-T', 'fields',
    '-e', 'dns.qry_name'
]

result = subprocess.run(cmd, capture_output=True, text=True)

if result.returncode != 0:
    print(" Error running tshark:", result.stderr)
else:
    domains = result.stdout.strip().split('\n')
    domain_counts = Counter(domain.strip() for domain in domains if domain.strip())

    print(" Top 10 Most Queried Domains:")
    print("=" * 50)
    for domain, count in domain_counts.most_common(10):
        print(f"{domain}: {count} queries")


 Error running tshark: tshark: Some fields aren't valid:
	dns.qry_name



In [14]:
import pyshark

# Choose the correct network interface (change this if needed)
# To find interface names: pyshark.LiveCapture().interfaces
interface = 'Wi-Fi'  # or 'Ethernet', 'en0', etc.

print(f"📡 Starting Live Capture on interface: {interface}")

# Capture live packets on TCP port 80 (HTTP traffic)
capture = pyshark.LiveCapture(interface=interface, display_filter='tcp.port == 80')

packet_count = 0
max_packets = 10

try:
    for packet in capture.sniff_continuously():
        packet_count += 1
        print(f"\n📦 Packet #{packet_count}")
        print(f"▶ Source: {packet.ip.src}")
        print(f"▶ Destination: {packet.ip.dst}")
        print(f"▶ Protocol: {packet.transport_layer}")
        print(f"▶ Length: {packet.length}")
        
        if 'http' in packet:
            try:
                http_layer = packet.http
                print(f"🌐 HTTP Host: {http_layer.host}")
                print(f"🌐 URI: {http_layer.request_uri}")
                print(f"🌐 Full URL: http://{http_layer.host}{http_layer.request_uri}")
            except AttributeError:
                print("HTTP packet, but missing fields.")
        
        if packet_count >= max_packets:
            break

except KeyboardInterrupt:
    print("✋ Capture interrupted.")
except Exception as e:
    print(f"❌ Error: {e}")
finally:
    capture.close()
    print("✅ Capture complete.")


📡 Starting Live Capture on interface: Wi-Fi
❌ Error: Cannot run the event loop while another loop is running


C:\Users\Sakthi\AppData\Local\Temp\ipykernel_23188\824252639.py:39: RuntimeWarning: coroutine 'LiveCapture._get_tshark_process' was never awaited
  print(f"❌ Error: {e}")


RuntimeError: Cannot run the event loop while another loop is running